In [ ]:
! pip install selenium
! pip install undetected_chromedriver
! pip install python-dotenv

In [3]:
import os
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.select import Select
from dotenv import load_dotenv

In [ ]:
u0 = 'https://www.wanted.co.kr/salary/655' # 3,613
# u1 = 'https://www.wanted.co.kr/salary/655' # 3,773 -> url이 같아서 의미없음 -> 동적크롤링만이 답..?
# 야매로는 그냥 각 막대그래프의 height값으로 연봉 비교할 수 있긴함;;
# 도대체 구조가 어떻게 되어있는 것인가?
# -> 근데 또, html의 모든 값이 불러와지지 않음 -> ex)recharts클래스를 가진 path태그가 제대로 출력 안됨
    # 이유: 아래에서 lxml로 파싱했기 때문 -> html형식으로 파싱하면 됨
    # 아님. 정적크롤링으로는 안됨 https://www.inflearn.com/questions/1031893/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91-requests%EB%A5%BC-%ED%95%98%EB%A9%B4-%EC%9D%BC%EB%B6%80-%EC%86%8C%EC%8A%A4%EC%BD%94%EB%93%9C%EB%A7%8C-%EC%B6%9C%EB%A0%A5%EC%9D%B4-%EB%90%A9%EB%8B%88%EB%8B%A4
    # -> 따라서, 셀레니움으로 가져와야 함
    # +이유: JS로 실행되는 페이징을 크롤링 하기 위해서는 selenium 라이브러리를 사용하여 파이썬 코드에서 직접 화면의 JS를 실행시켜야 한다
a = requests.get(u0).text
a1 = bs(a, 'html') # a1 = bs(a, "lxml")
# print(a1.prettify())

a1.select('g.recharts') # [], 안나옴
# a1.select('h2.Info_jobCategorySalary__M4Ql6')[0].text # ' 3,613  만원'

In [64]:
# 아래처럼 하면 팝업창(사이트 권한 허용 관련)은 안뜸
# 대신, webdriver객체기 때문에 구글 로그인은 안됨..
# 구글 로그인 권한 푸는것하고 사이트 권한 푸는건 동시에 안되는듯..

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--disable-notifications')  # Disable notifications 

# wd = webdriver.Chrome(options=chrome_options)
# wd.get('https://www.wanted.co.kr/salary/655')

In [5]:
# (1)권한 수락 팝업창떠서, 그걸 빠르게 허용을 눌러줘야 데이트가 제대로 뜸
    # ㄴㄴ 잘못 알고 있었음. 그냥, time.sleep()수치를 제대로 안 줘서 안 뜨는 것
    # 근데, 일반 webdriver.Chrome()객체로 하면 팝업창 수락 작동하는데,
    # undetected_chromedriver객체 쓰면 안됨

driver = uc.Chrome()
driver.get('https://www.wanted.co.kr/salary/518')
time.sleep(3)

select1 = '#__next > div.Salary_Salary__OUYDA > div > div.SearchBar_SearchBar__8ma00 > div > div > div:nth-child(3) > div > select'
driver.find_element(By.CSS_SELECTOR, select1).click()
time.sleep(3)

select2 = '#__next > main > section > div > div > div > form > div.css-1qsnvsb > button.css-you85z > p'
driver.find_element(By.CSS_SELECTOR, select2).click()
time.sleep(3)


load_dotenv() # 이 함수를 한번 실행시켜야, .env파일에서 가져오기 가능
# google_id
driver.find_element(By.CSS_SELECTOR, '#identifierId').click()
driver.find_element(By.CSS_SELECTOR, '#identifierId').send_keys(os.getenv('ID'))
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#identifierNext > div > button').click()
time.sleep(4)
# google_pw
driver.find_element(By.CSS_SELECTOR, '#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input').click()
driver.find_element(By.CSS_SELECTOR, '#password > div.aCsJod.oJeWuf > div > div.Xb9hP > input').send_keys(os.getenv('PW'))
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#passwordNext > div > button').click()
time.sleep(10)

# 3. select태그 안에 있는 것들 어떻게 선택? 일일이 select click시키면 안됨
    # https://hoood.tistory.com/1973
        # way1: select.select_by_index(1) # 자바 개발자
        # way2: select.select_by_visible_text('데이터 엔지니어'
        # way3: select.select_by_value('677')
    # driver.find_element(By.CSS_SELECTOR, '#..').click()     ... timesleep(3)
        # subCategory버튼을 click하는 코드 필요없음 -> 아래 select문으로도 충분함
    # 총 개수는 38 -> 0~37 xxxx -> 1~38: idx라도 이건 걍 1부터 시작하나봄ㅋㅋㅋㅋㅋㅋ 환장하겠네
        # 아. 아님. 0인덱스는 '직무'로 block처리된거임. 0부터 시작하는게 맞긴 하네

# ? (+)흠;; 데이터를 또 어떻게 담을지가 고민이네
    # dict에 key를 직무별, value값들은 0(신입)~10년차를 리스트로 담으면 데이터 정규화법칙에 위배됨
    # dict를 2중 구조로 만들면 될..듯. -> 는 무슨.. dict로 할당된 value값도 결국엔 여러개 요소를 가진 것임
    # 'a직무0년차', 'a직무1년차' ... 'z직무10년차' 이렇게 하지 않는 이상 해결은 안 될듯
    # 일단은 그냥 이중dict로 만들기 - db에 저장하는게 아니니까, 치명적인 실수는 아니라고 생각함ㅇㅇ.
        # 데이터 조회해서 db.join이런거 할거 아니잖아

select = Select(driver.find_element(By.NAME, 'subCategory'))
all_options = select.options
all_options[0].text
salary_dict = {}
for k in range(1,len(all_options)): # 0은 '직무' block태그임
    # step1. 조회
    select.select_by_index(k) 
    key = all_options[k].text
    time.sleep(3)

    # step2. 신입(0년) 연봉 데이터 추출
    soup = bs(driver.page_source, 'html.parser') # xlml이 아니라, html.parser로 해야 안 짤리고 전부 나옴
    newbie = soup.select('h2.Info_jobCategorySalary__M4Ql6')[0].text # 신입연봉: ' x,xxx  만원'
    newbie = int(newbie[1:6].replace(',','')) # int: 3613
    # *time.sleep(3) 필요없나?

    # step2-2. 신입,1~10 -> 신입의 height기준으로 나머지 비율구해서 연봉구하기 can
    len(soup.select('path.recharts-rectangle')) # 11
    path_list = soup.select('path.recharts-rectangle')
    height_list = [x['height'] for x in path_list]
    height_list # 신입~10년차 막대의 height가 주어짐
    salary_list = [int(float(y)*newbie/float(height_list[0])) for y in height_list]
    # [3613, 3772, 4138, 4557, 4984, 5041, 5461, 6054, 6426, 6513, 7698]
    value = salary_list
    salary_dict[key] = value

print(salary_dict)
# {'전체': [3535, 3693, 3982, 4316, 4623, 4818..], ...,  'DBA': [3535, 3693, ..., 5859, 6185, 7056]}

{'전체': [3535, 3693, 3982, 4316, 4623, 4818, 5176, 5499, 5859, 6185, 7056], '데이터 엔지니어': [3613, 3772, 4138, 4557, 4984, 5041, 5461, 6054, 6426, 6513, 7698], '임베디드 개발자': [3517, 3674, 3832, 4136, 4651, 4810, 5120, 5557, 5713, 6177, 6918], '자바 개발자': [3354, 3599, 3839, 4203, 4434, 4748, 5084, 5227, 5639, 6016, 6860], '.NET 개발자': [2974, 3286, 3545, 4003, 4297, 4418, 4919, 5122, 5356, 5590, 6506], '시스템,네트워크 관리자': [3470, 3546, 3633, 4083, 4316, 4461, 5038, 5374, 5519, 5922, 6654], '프론트엔드 개발자': [3511, 3659, 3874, 4169, 4443, 4715, 5091, 5214, 5672, 6187, 6704], '보안 엔지니어': [3043, 3371, 3831, 4206, 4478, 4790, 4961, 5270, 5473, 5677, 6659], '하드웨어 엔지니어': [3415, 3564, 3714, 3864, 4205, 4468, 4731, 5069, 5407, 5745, 6082], 'DevOps / 시스템 관리자': [3713, 3833, 4065, 4419, 4822, 4998, 5620, 5937, 6098, 6400, 7558], 'QA,테스트 엔지니어': [3075, 3287, 3378, 3736, 4113, 4123, 4308, 4677, 4719, 4760, 5893], '안드로이드 개발자': [3574, 3626, 3867, 4137, 4579, 4715, 5096, 5366, 5634, 6126, 6778], 'iOS 개발자': [3435, 3681, 4127, 